# 线性回归的简洁实现

## 1. 生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 2. 调用API 来读取数据集

In [2]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个Pytorch 数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    # DataLoader 随机抽取小批量的数据， shuffle 打乱顺序
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[-1.0518,  0.0756],
         [-0.5473,  0.4023],
         [-0.8209,  2.8097],
         [ 0.4005, -0.9243],
         [-0.2954,  1.2515],
         [-0.7147, -0.6349],
         [-1.3398, -0.3372],
         [-1.7104,  0.5128],
         [ 0.4626, -1.1017],
         [-0.6894, -0.2257]]),
 tensor([[ 1.8454],
         [ 1.7237],
         [-6.9926],
         [ 8.1443],
         [-0.6439],
         [ 4.9497],
         [ 2.6654],
         [-0.9584],
         [ 8.8666],
         [ 3.5751]])]

# 3.定义模型

In [3]:
# nn 是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

# 4. 初始化模型参数

In [4]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 5. 定义损失函数

In [5]:
# 平均 L_2 范数
loss = nn.MSELoss()

## 6.定义优化算法

In [6]:
# 实例化SGD算法
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 7. 训练

In [7]:
num_epochs = 3

for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        # 清零
        trainer.zero_grad()
        l.backward()
        # 模型更新
        trainer.step()

    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000261
epoch 2, loss 0.000100
epoch 3, loss 0.000101


## 8.评估结果

In [8]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0005, -0.0014])
b的估计误差： tensor([-0.0006])
